In [1]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 
device = "cuda"

In [2]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler

In [3]:
import import_ipynb

In [4]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        self.transforms = transforms.ToTensor()
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        x = self.x[index, ...]
        y = self.y[index, ...]
        return x, y
    
    def __len__(self):
        return len(self.x)

In [5]:
def Normalzie(data):
    normals = []
    scaler = StandardScaler()
    for idx in range(len(data)):
        normals.append(scaler.fit_transform(data[idx]))
    return np.array(normals)

In [6]:
PNES_data = mne.read_epochs(r"D:\MNE Data\PNES\RostamiAlireza.fif", preload=False).get_data(picks='eeg');
TLE_data = mne.read_epochs(r"D:\MNE Data\TLE\AlipoorMohamadHakim.fif", preload=False).get_data(picks='eeg');

Reading D:\MNE Data\PNES\RostamiAlireza.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.67 ms
        0 CTF compensation matrices available
Not setting metadata
693 matching events found
No baseline correction applied
0 projection items activated
Loading data for 693 events and 3000 original time points ...
Reading D:\MNE Data\TLE\AlipoorMohamadHakim.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.67 ms
        0 CTF compensation matrices available
Not setting metadata
650 matching events found
No baseline correction applied
0 projection items activated
Loading data for 650 events and 3000 original time points ...


In [7]:
PNES_data = Normalzie(PNES_data)
TLE_data = Normalzie(TLE_data)

In [8]:
PNES_labels = np.zeros((PNES_data.shape[0]))
TLE_labels = np.ones((TLE_data.shape[0]))

In [9]:
data = np.concatenate((PNES_data, TLE_data), axis=0).reshape(-1, 1, 19, 3000)
labels = np.concatenate((PNES_labels, TLE_labels))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [19]:
class network(nn.Module):
    def __init__(self, batch_size, T, C, input_size, hidden_size, num_layers):
        super(network, self).__init__()
        self.N = batch_size
        self.T = T
        self.C = C

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.cell_count = self.T // self.input_size


        self._lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)

        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])


    def forward(self, x):

        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.cell_count, self.input_size)
        _x = None

        for index, cell in enumerate(self.lstm):
            cell_out, _ = cell(x[:, index, :, :], None)
            last_layer_out = cell_out[:, -1, :]
            
            last_layer_out.unsqueeze(0)
            if _x is None:
                _x = last_layer_out
            else:
                _x = torch.cat((_x, last_layer_out), dim=0)
            
        _x = _x.reshape(self.C, -1, self.hidden_size)
        x = _x.permute(1, 0, 2).unsqueeze(1)

        return x

In [20]:
b = torch.Tensor(479, 3000, 19)
a = torch.rand(479, 1, 19, 3000)

m = network(batch_size=479, T = 3000, C = 19, input_size = 60, hidden_size = 30, num_layers=4)
m(a).shape

torch.Size([479, 1, 19, 30])

In [ ]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x